# FinSense — CFO KPI & Sentiment Extraction (Multi-Company)

This notebook takes the parsed transcript segments from `data/processed/transcripts.csv`,
filters **CFO prepared remarks**, and extracts:

- basic KPIs (e.g., revenue YoY growth if mentioned)
- sentiment (polarity, subjectivity)
- metadata (company, fiscal period, doc path)

It then builds a `cfo_insights_enriched` DataFrame and writes JSON "insight packs"
for each CFO segment into `data/insights/`.

In [1]:
# ============================================================
# Cell 1 – Imports & paths
# ============================================================

import json
from pathlib import Path
import re

import pandas as pd
from textblob import TextBlob  # already in requirements

# Paths relative to this notebook
TRANSCRIPTS_PATH = Path("../data/processed/transcripts.csv")
WATCHLIST_PATH = Path("../configs/watchlist.csv")
INSIGHTS_DIR = Path("../data/insights")

print("Transcripts path:", TRANSCRIPTS_PATH.resolve())
print("Watchlist path:", WATCHLIST_PATH.resolve())
print("Insights dir:", INSIGHTS_DIR.resolve())


Transcripts path: /Users/aravindanisetti/Desktop/2025 Training/projects finance/finsense-earnings-ai/data/processed/transcripts.csv
Watchlist path: /Users/aravindanisetti/Desktop/2025 Training/projects finance/finsense-earnings-ai/configs/watchlist.csv
Insights dir: /Users/aravindanisetti/Desktop/2025 Training/projects finance/finsense-earnings-ai/data/insights


In [2]:
# ============================================================
# Cell 2 – Load transcripts + watchlist
# ============================================================

df = pd.read_csv(TRANSCRIPTS_PATH)
print("Loaded transcripts:", df.shape)
display(df.head())

# Make sure some expected columns exist
expected_cols = {
    "doc_path",
    "company_hint",
    "fiscal_year",
    "fiscal_quarter",
    "ingest_date",
    "segment_index",
    "speaker",
    "section",
    "text",
    "source",
}
missing = expected_cols - set(df.columns)
if missing:
    print("[WARN] Missing expected columns in transcripts:", missing)

# Optional: load watchlist for ticker → sector / company_name
ticker_to_sector = {}
ticker_to_name = {}

if WATCHLIST_PATH.exists():
    wl = pd.read_csv(WATCHLIST_PATH)
    if {"ticker", "sector", "company_name"}.issubset(wl.columns):
        ticker_to_sector = dict(zip(wl["ticker"].str.upper(), wl["sector"]))
        ticker_to_name = dict(zip(wl["ticker"].str.upper(), wl["company_name"]))
        print(f"Loaded watchlist tickers: {len(ticker_to_sector)}")
    else:
        print("[WARN] Watchlist present but lacking expected columns.")
else:
    print("[INFO] No watchlist.csv found – sector / clean company names may show as 'Unknown'.")


Loaded transcripts: (136, 10)


,doc_path,company_hint,fiscal_year,fiscal_quarter,ingest_date,segment_index,speaker,section,text,source
0,data/raw/NFLX_2024Q2_SAMPLE_REMARKS.txt,NFLX 2024Q2 SAMPLE REMARKS,2024.0,Q2,2025-11-24,0,FULL_TEXT,prepared_remarks,Prepared Remarks – CFO\n\nRevenue grew 12% yea...,manual_drop
1,data/raw/AMD_2024Q2_SAMPLE_REMARKS.txt,AMD 2024Q2 SAMPLE REMARKS,2024.0,Q2,2025-11-24,0,FULL_TEXT,prepared_remarks,Prepared Remarks – CFO\n\nRevenue increased 8%...,manual_drop
2,data/raw/ADBE_2024Q2_SAMPLE_REMARKS.txt,ADBE 2024Q2 SAMPLE REMARKS,2024.0,Q2,2025-11-24,0,FULL_TEXT,prepared_remarks,Prepared Remarks – CFO\n\nTotal revenue increa...,manual_drop
3,data/raw/NVDA_2024Q2_Remarks.txt,NVDA 2024Q2 Remarks,2024.0,Q2,2025-11-24,0,OPERATOR,prepared_remarks,Welcome to Q2 2024 results.,manual_drop
4,data/raw/NVDA_2024Q2_Remarks.txt,NVDA 2024Q2 Remarks,2024.0,Q2,2025-11-24,1,CFO,prepared_remarks,Revenue grew 10% year-over-year.,manual_drop


Loaded watchlist tickers: 20


In [3]:
# ============================================================
# Cell 3 – Helper: basic KPI extraction from CFO text
# ============================================================

def _find_pct(pattern: str, text: str):
    """Return first percentage number matched in text (as float), else None."""
    m = re.search(pattern, text, flags=re.I | re.S)
    if not m:
        return None
    try:
        return float(m.group(1))
    except Exception:
        return None


def extract_basic_kpis(text: str) -> dict:
    """
    Very lightweight KPI extraction for demo:
      - revenue_growth_yoy_pct
      - eps_growth_yoy_pct
      - guidance_comment
      - margin_comment
    """
    if not isinstance(text, str):
        text = str(text or "")

    # Revenue YoY %
    revenue_growth = _find_pct(
        r"(?:revenue|sales)[^%]{0,60}?(\d{1,2}\.?\d*)\s*%[^%]{0,40}?(?:year[- ]over[- ]year|yoy)",
        text,
    )

    # EPS YoY %
    eps_growth = _find_pct(
        r"(?:EPS|earnings per share)[^%]{0,60}?(\d{1,2}\.?\d*)\s*%[^%]{0,40}?(?:year[- ]over[- ]year|yoy)",
        text,
    )

    # Guidance + margins – just simple keyword flags
    t_low = text.lower()

    guidance_comment = None
    if "guidance" in t_low:
        if any(w in t_low for w in ["raise", "raised", "increase", "increased", "above", "upward"]):
            guidance_comment = "guidance raised / positive"
        elif any(w in t_low for w in ["lower", "lowered", "reduce", "reduced", "downward", "cut"]):
            guidance_comment = "guidance lowered / cautious"
        else:
            guidance_comment = "guidance mentioned"

    margin_comment = None
    if "margin" in t_low:
        if any(w in t_low for w in ["expand", "expanded", "improve", "improved", "strong"]):
            margin_comment = "margin commentary positive"
        elif any(w in t_low for w in ["compress", "compressed", "pressure", "down", "weaker"]):
            margin_comment = "margin commentary negative"
        else:
            margin_comment = "margin commentary present"

    return {
        "revenue_growth_yoy_pct": revenue_growth,
        "eps_growth_yoy_pct": eps_growth,
        "guidance_comment": guidance_comment,
        "margin_comment": margin_comment,
    }

# Quick sanity check on a known CFO line, if you want:
sample_rows = df[df["speaker"].str.upper().eq("CFO")].head(1)
if not sample_rows.empty:
    s_text = sample_rows.iloc[0]["text"]
    print("Sample CFO text snippet:\n", s_text[:260], "...\n")
    print("Extracted KPIs:", extract_basic_kpis(s_text))


Sample CFO text snippet:
 Revenue grew 10% year-over-year. ...

Extracted KPIs: {'revenue_growth_yoy_pct': 10.0, 'eps_growth_yoy_pct': None, 'guidance_comment': None, 'margin_comment': None}


In [4]:
# ============================================================
# Cell 4 – Filter to CFO / FULL_TEXT prepared remarks
# ============================================================

# Normalize speaker / section
df["speaker_norm"] = df["speaker"].astype(str).str.upper()
df["section_norm"] = df["section"].astype(str).str.lower()

# Keep only CFO or FULL_TEXT and prepared remarks / preface-like sections
candidates = df[
    df["speaker_norm"].isin(["CFO", "FULL_TEXT"])
    & df["section_norm"].isin(["prepared_remarks", "preface", "full_text"])
].copy()

print("Total segments after CFO/FULL_TEXT + section filter:", len(candidates))
display(candidates[["doc_path", "company_hint", "fiscal_year", "fiscal_quarter", "speaker", "section"]].head(10))


Total segments after CFO/FULL_TEXT + section filter: 6


,doc_path,company_hint,fiscal_year,fiscal_quarter,speaker,section
0,data/raw/NFLX_2024Q2_SAMPLE_REMARKS.txt,NFLX 2024Q2 SAMPLE REMARKS,2024.0,Q2,FULL_TEXT,prepared_remarks
1,data/raw/AMD_2024Q2_SAMPLE_REMARKS.txt,AMD 2024Q2 SAMPLE REMARKS,2024.0,Q2,FULL_TEXT,prepared_remarks
2,data/raw/ADBE_2024Q2_SAMPLE_REMARKS.txt,ADBE 2024Q2 SAMPLE REMARKS,2024.0,Q2,FULL_TEXT,prepared_remarks
4,data/raw/NVDA_2024Q2_Remarks.txt,NVDA 2024Q2 Remarks,2024.0,Q2,CFO,prepared_remarks
12,data/raw/NFLX_2025Q4_Content_Accounting_Overvi...,NFLX 2025Q4 Content Accounting Overview,2025.0,Q4,FULL_TEXT,prepared_remarks
125,data/raw/NFLX_Content Accounting Overview.pdf,NFLX Content Accounting Overview,NaN,NaN,FULL_TEXT,prepared_remarks


## Filter to CFO Prepared Remarks

We only want the CFO's **prepared remarks** segments, which are usually where
high-level KPIs and guidance comments are given.


In [5]:
# ============================================================
# Cell 5 – Drop non-earnings docs & rows missing year/quarter
# ============================================================

# 1) Drop anything without fiscal year or quarter (this is what causes '?? ??' entries)
before = len(candidates)
candidates = candidates.dropna(subset=["fiscal_year", "fiscal_quarter"])
candidates = candidates[candidates["fiscal_quarter"].isin(["Q1", "Q2", "Q3", "Q4"])]
print(f"Dropped {before - len(candidates)} rows with missing/invalid year or quarter.")

# 2) Drop obvious non-earnings docs based on filename keywords
NON_EARNINGS_KEYWORDS = [
    "SUSTAINABILITY",
    "CONTENT_ACCOUNTING",
    "CONTENT ACCOUNTING",
    "OVERVIEW",
    "PRESENTATION",
    "PRESS_RELEASE",
    "PRESS RELEASE",
    "SUPPLEMENTAL",
]

upper_paths = candidates["doc_path"].astype(str).str.upper()
mask_non = upper_paths.str.contains("|".join(NON_EARNINGS_KEYWORDS), regex=True)
print("Non-earnings rows detected:", mask_non.sum())

candidates = candidates[~mask_non].copy()
print("Segments after removing non-earnings docs:", len(candidates))

display(
    candidates[
        ["doc_path", "company_hint", "fiscal_year", "fiscal_quarter", "speaker", "section"]
    ].head(15)
)


Dropped 1 rows with missing/invalid year or quarter.
Non-earnings rows detected: 1
Segments after removing non-earnings docs: 4


,doc_path,company_hint,fiscal_year,fiscal_quarter,speaker,section
0,data/raw/NFLX_2024Q2_SAMPLE_REMARKS.txt,NFLX 2024Q2 SAMPLE REMARKS,2024.0,Q2,FULL_TEXT,prepared_remarks
1,data/raw/AMD_2024Q2_SAMPLE_REMARKS.txt,AMD 2024Q2 SAMPLE REMARKS,2024.0,Q2,FULL_TEXT,prepared_remarks
2,data/raw/ADBE_2024Q2_SAMPLE_REMARKS.txt,ADBE 2024Q2 SAMPLE REMARKS,2024.0,Q2,FULL_TEXT,prepared_remarks
4,data/raw/NVDA_2024Q2_Remarks.txt,NVDA 2024Q2 Remarks,2024.0,Q2,CFO,prepared_remarks


## KPI Extraction Helper

We define a simple **rule-based extractor** that looks for:

- revenue YoY growth patterns like: `"10% year-over-year"`
- EPS growth patterns like: `"EPS grew 5%"`
- presence of words like `"guidance"`, `"outlook"`, `"forecast"`, `"margin"`

This is intentionally lightweight — enough to demonstrate signal extraction
for portfolio / credit analytics, without being a full NLP engine.


In [6]:
# ============================================================
# Cell 6 – Metadata repair: ticker, company name, sector
# ============================================================

def repair_metadata(row):
    """Fill in ticker, company_hint, sector where possible."""
    # --- Ticker ---
    ticker = row.get("ticker") if "ticker" in row else None
    if pd.isna(ticker) or not ticker:
        # infer from filename
        fname = Path(str(row["doc_path"])).name.upper()
        guess = fname.split("_")[0]
        if guess.isalpha() and 2 <= len(guess) <= 6:
            ticker = guess
        else:
            ticker = "UNKNOWN"
    else:
        ticker = str(ticker).upper()
    row["ticker"] = ticker

    # --- Company name ---
    company_hint = row.get("company_hint") or ""
    if (
        (not company_hint)
        or (company_hint.upper().startswith("SAMPLE"))
        or (company_hint.strip() in ["", "-", "--"])
    ):
        # Prefer watchlist company_name if we have it
        if ticker_to_name:
            row["company_hint"] = ticker_to_name.get(ticker, company_hint or ticker)
        else:
            row["company_hint"] = company_hint or ticker
    # else: keep existing company_hint

    # --- Sector ---
    if ticker_to_sector:
        row["sector"] = ticker_to_sector.get(ticker, "Unknown")
    else:
        # ensure the column exists
        if "sector" not in row or pd.isna(row["sector"]):
            row["sector"] = "Unknown"

    return row


clean_df = candidates.apply(repair_metadata, axis=1)

print("After metadata repair:")
display(clean_df[["doc_path", "ticker", "company_hint", "sector", "fiscal_year", "fiscal_quarter"]].head(15))


After metadata repair:


,doc_path,ticker,company_hint,sector,fiscal_year,fiscal_quarter
0,data/raw/NFLX_2024Q2_SAMPLE_REMARKS.txt,NFLX,NFLX 2024Q2 SAMPLE REMARKS,Media,2024.0,Q2
1,data/raw/AMD_2024Q2_SAMPLE_REMARKS.txt,AMD,AMD 2024Q2 SAMPLE REMARKS,Tech,2024.0,Q2
2,data/raw/ADBE_2024Q2_SAMPLE_REMARKS.txt,ADBE,ADBE 2024Q2 SAMPLE REMARKS,Tech,2024.0,Q2
4,data/raw/NVDA_2024Q2_Remarks.txt,NVDA,NVDA 2024Q2 Remarks,Tech,2024.0,Q2


In [7]:
# ============================================================
# Cell 7 – Build a single insight pack from a row
# ============================================================

def make_insight_pack(row: pd.Series) -> dict:
    """Create a JSON-serializable insight pack for one CFO/full-text segment."""
    text = row.get("text", "") or ""

    kpis = extract_basic_kpis(text)
    sentiment = TextBlob(text).sentiment  # (polarity, subjectivity)

    # Cast numeric types to plain Python for JSON
    fy = row.get("fiscal_year")
    fq = row.get("fiscal_quarter")
    seg_idx = row.get("segment_index")

    try:
        fiscal_year = int(fy) if pd.notna(fy) else None
    except Exception:
        fiscal_year = None

    fiscal_quarter = str(fq) if fq and fq in ["Q1", "Q2", "Q3", "Q4"] else None

    try:
        seg_idx_int = int(seg_idx) if pd.notna(seg_idx) else 0
    except Exception:
        seg_idx_int = 0

    ticker = row.get("ticker")
    company_hint = row.get("company_hint")
    sector = row.get("sector", "Unknown")

    # Core pack
    pack = {
        "ticker": ticker,
        "company_hint": company_hint,
        "sector": sector,
        "fiscal_year": fiscal_year,
        "fiscal_quarter": fiscal_quarter,
        "speaker": row.get("speaker", row.get("speaker_norm", "CFO")),
        "section": row.get("section", row.get("section_norm", "")),
        "kpis": kpis,
        "sentiment": {
            "polarity": float(sentiment.polarity),
            "subjectivity": float(sentiment.subjectivity),
        },
        # Include raw text so the UI preview looks professional
        "raw_text": text,
        "meta": {
            "doc_path": row.get("doc_path"),
            "ingest_date": row.get("ingest_date"),
            "source": row.get("source"),
            "segment_index": seg_idx_int,
        },
    }

    return pack

# Quick sanity check on the first row
test_row = clean_df.head(1).iloc[0]
test_pack = make_insight_pack(test_row)
print(json.dumps(test_pack, indent=2)[:800], "...\n")


{
  "ticker": "NFLX",
  "company_hint": "NFLX 2024Q2 SAMPLE REMARKS",
  "sector": "Media",
  "fiscal_year": 2024,
  "fiscal_quarter": "Q2",
  "speaker": "FULL_TEXT",
  "section": "prepared_remarks",
  "kpis": {
    "revenue_growth_yoy_pct": 12.0,
    "eps_growth_yoy_pct": null,
    "guidance_comment": "guidance mentioned",
    "margin_comment": "margin commentary positive"
  },
  "sentiment": {
    "polarity": 0.1355050505050505,
    "subjectivity": 0.4824242424242424
  },
  "raw_text": "Prepared Remarks \u2013 CFO\n\nRevenue grew 12% year-over-year, driven by steady subscriber growth and higher ARPU.\nOperating margin expanded 150 basis points as content amortization normalized.\nContent spend grew modestly, but we remain focused on disciplined returns on new titles.\nWe are raising full- ...



In [8]:
# ============================================================
# Cell 8 – Write all insight packs to ../data/insights
# ============================================================

INSIGHTS_DIR.mkdir(parents=True, exist_ok=True)

written = 0
skipped = 0

for _, row in clean_df.iterrows():
    pack = make_insight_pack(row)

    # If we still somehow lack year/quarter, skip writing to avoid '??' in UI
    if pack["fiscal_year"] is None or pack["fiscal_quarter"] is None:
        skipped += 1
        continue

    ticker = (pack["ticker"] or "UNKNOWN").upper()
    year = pack["fiscal_year"]
    quarter = pack["fiscal_quarter"]
    seg = pack["meta"]["segment_index"]

    filename = f"{ticker}_{year}{quarter}_seg{seg}.json"
    out_path = INSIGHTS_DIR / filename

    with out_path.open("w", encoding="utf-8") as f:
        json.dump(pack, f, indent=2)

    written += 1

print(f"Wrote {written} insight packs to {INSIGHTS_DIR}")
print(f"Skipped {skipped} rows without clean year/quarter.")


Wrote 4 insight packs to ../data/insights
Skipped 0 rows without clean year/quarter.


## Sentiment Extraction Helper

We use a lightweight sentiment model (`TextBlob`) to get:

- **polarity**: [-1, 1]
- **subjectivity**: [0, 1]

This isn't finance-specific, but it's enough to demonstrate how FinSense can
attach directional "tone" to CFO commentary.


In [10]:
import pandas as pd

df = pd.read_csv("data/processed/transcripts.csv")

cfo_mask = df["speaker"].str.upper().eq("CFO")
remarks_mask = df["section"].isin(["FULL_TEXT", "prepared_remarks"])

df_cfo = df[cfo_mask & remarks_mask]
df_cfo[["company_hint", "fiscal_year", "fiscal_quarter"]].drop_duplicates()


FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/transcripts.csv'

## Build `cfo_insights_enriched` DataFrame

Now we loop over all CFO prepared-remarks segments and create an enriched table with:

- company & period metadata  
- raw text location (`doc_path`)  
- extracted KPIs  
- sentiment scores


written = 0

for idx, row in cfo_segments.iterrows():
    pack = make_insight_pack(row)

    company_slug = str(pack["company_hint"]).split()[0].upper()  # AMD, ADBE, NFLX, etc.
    year = pack["fiscal_year"] or "NA"
    quarter = pack["fiscal_quarter"] or "NA"
    seg = pack["meta"]["segment_index"]

    out_name = f"cfo_insight_pack_{company_slug}_{year}{quarter}_seg{seg}.json"
    out_path = INSIGHTS_DIR / out_name

    with out_path.open("w", encoding="utf-8") as f:
        json.dump(pack, f, indent=2)

    written += 1

print(f"Wrote {written} CFO insight packs to {INSIGHTS_DIR}")


sorted(p.name for p in INSIGHTS_DIR.glob("cfo_insight_pack_*.json"))


## Write JSON "Insight Packs" for Each CFO Segment

For downstream systems (dashboards, credit analytics, portfolio tools), we
export each CFO prepared-remarks segment as a **single JSON object** with:

- metadata (company, fiscal period, doc path)
- KPI fields
- sentiment
- basic provenance

Files are written to `../data/insights/` as:

`cfo_insight_<index>.json` for now (you can later shift to ticker/quarter naming).


# If company_hint is literally "AMD" use that; 
# otherwise we search for rows where doc_path contains "AMD".
amd_rows = cfo_prepared[cfo_prepared["company_hint"].str.contains("AMD", case=False, na=False)]

if amd_rows.empty:
    amd_rows = cfo_prepared[cfo_prepared["doc_path"].str.contains("AMD", case=False, na=False)]

amd_rows[["company_hint", "fiscal_year", "fiscal_quarter", "doc_path"]].head()


if amd_rows.empty:
    raise ValueError("No AMD CFO prepared-remarks segment found. Check transcripts.csv.")

amd_row = amd_rows.iloc[0]

amd_insight = make_insight_pack(amd_row)
amd_insight


out_dir = Path("../data/insights")
out_dir.mkdir(parents=True, exist_ok=True)

out_path = out_dir / "cfo_insight_pack_AMD_2025Q4.json"

with out_path.open("w", encoding="utf-8") as f:
    json.dump(amd_insight, f, indent=2)

print("Wrote AMD insight pack to:", out_path)